In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# !pip install tensorflow
# !pip install keras
# !pip install pandas
# !pip install matplotlib
# !pip install numpy
# !pip install seaborn
# !pip install sklearn
# !pip install glob
# !pip install cv2
# !pip install plotly

In [ ]:
import pandas as pd       
import matplotlib as mat
import matplotlib.pyplot as plt  
import matplotlib.image as mpimg   
import numpy as np
import seaborn as sns
%matplotlib inline

pd.options.display.max_colwidth = 100

import random
import os

from numpy.random import seed
seed(42)

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from keras.metrics import AUC, Accuracy, Precision, SensitivityAtSpecificity
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import glob

from tensorflow.random import set_seed
set_seed(42)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import time
initial_time = time.time() #Time for the notebook starting to run

In [ ]:
IMG_SIZE = 224
BATCH = 32
SEED = 42
IMG_SIZE_EFF=229
epos = 30 # Number of epochs to train
oversampling_flag = False

In [ ]:
main_path = "C:/Users/nourn/Desktop/post_grad/winter/machine learning/project/pnemonia/code/data/chest_xray/chest_xray/"
train_path = os.path.join(main_path,"train")
test_path=os.path.join(main_path,"test")

train_normal = glob.glob(train_path+"/NORMAL/*.jpeg")
train_pneumonia = glob.glob(train_path+"/PNEUMONIA/*.jpeg")

test_normal = glob.glob(test_path+"/NORMAL/*.jpeg")
test_pneumonia = glob.glob(test_path+"/PNEUMONIA/*.jpeg")

In [ ]:
train_list = [x for x in train_normal]
train_list.extend([x for x in train_pneumonia])

df_train = pd.DataFrame(np.concatenate([['Normal']*len(train_normal) , ['Pneumonia']*len(train_pneumonia)]), columns = ['class'])
df_train['image'] = [x for x in train_list]

test_list = [x for x in test_normal]
test_list.extend([x for x in test_pneumonia])

df_test = pd.DataFrame(np.concatenate([['Normal']*len(test_normal) , ['Pneumonia']*len(test_pneumonia)]), columns = ['class'])
df_test['image'] = [x for x in test_list]

# <a id="3">Exploring the Data</a> 

Let's check the target distribution on each set

In [ ]:
plt.figure(figsize=(6,4))

ax = sns.countplot(x='class', data=df_train, palette="mako")

plt.xlabel("Class", fontsize= 12)
plt.ylabel("# of Samples", fontsize= 12)
plt.ylim(0,5000)
plt.xticks([0,1], ['Normal', 'Pneumonia'], fontsize = 11)

for p in ax.patches:
    ax.annotate((p.get_height()), (p.get_x()+0.30, p.get_height()+300), fontsize = 13)
    
plt.show()

In [ ]:
plt.figure(figsize=(7,5))

df_train['class'].value_counts().plot(kind='pie',labels = ['',''], autopct='%1.1f%%', colors = ['darkcyan','blue'], explode = [0,0.05], textprops = {"fontsize":15})

plt.legend(labels=['Pneumonia', 'Normal'])
plt.show()

In [ ]:
print('Train Set - Normal')

plt.figure(figsize=(12,12))

for i in range(0, 12):
    plt.subplot(3,4,i + 1)
    img = mpimg.imread(train_normal[i])
    plt.imshow(img,cmap='gray')
    plt.axis("off")

plt.tight_layout()

plt.show()

In [ ]:
print('Train Set - Pneumonia')

plt.figure(figsize=(12,12))

for i in range(0, 12):
    plt.subplot(3,4,i + 1)
    img = mpimg.imread(train_pneumonia[i])
    plt.imshow(img,cmap='gray')
    plt.axis("off")

plt.tight_layout()

plt.show()

# <a id="4">Preparing the Data</a> 

First, we need to create a validation set. To do that, we apply a simple stratified split on the original train dataset, using 80% for actual training and 20% for validation purposes.

In [ ]:
train_df, val_df = train_test_split(df_train, test_size = 0.20, random_state = SEED, stratify = df_train['class'])

## oversampling

In [ ]:
if oversampling_flag:
    df_normal = train_df[train_df['class'] =='Normal']
    df_pneumonia =  train_df[train_df['class'] =='Pneumonia']

    size_diff = len(df_normal) - len(df_pneumonia)
    # If the pneumonia class has fewer samples than normal class
    if size_diff > 0:
        # Upsample the pneumonia class to match the size of the normal class
        df_pneumonia_upsampled = resample(df_pneumonia, replace=True, n_samples=size_diff+len(df_pneumonia))
        # Concatenate the upsampled pneumonia class with the normal class
        train_df = pd.concat([df_normal, df_pneumonia_upsampled], ignore_index=True)
    else:
        # Upsample the normal class to match the size of the pneumonia class
        df_normal_upsampled = resample(df_normal, replace=True, n_samples=-size_diff+len(df_normal))
        # Concatenate the upsampled normal class with the pneumonia class
        train_df = pd.concat([df_pneumonia, df_normal_upsampled], ignore_index=True)

## dataloading
Now, we’re going to load the images from the folders and prepare them to feed our models. 

We begin by defining the data generators. With Keras Image Data Generator, we can rescale the pixel values and apply random transformation techniques for data augmentation on the fly. We define two different generators. The val_datagen is used to simply rescale the validation and test sets. The train_datagen includes some transformations to augment the train set.

We apply those generators on each dataset using the flow_from_dataframe method. Apart from the transformations defined in each generator, the images are also resized based on the target_size set.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255.,
                                  zoom_range = 0.1,
                                  #rotation_range = 0.1,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1)

val_datagen = ImageDataGenerator(rescale=1/255.)

ds_train = train_datagen.flow_from_dataframe(train_df,
                                             x_col = 'image',
                                             y_col = 'class',
                                             target_size = (IMG_SIZE, IMG_SIZE),
                                             class_mode = 'binary',
                                             batch_size = BATCH,
                                             seed = SEED)

ds_val = val_datagen.flow_from_dataframe(val_df,
                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'binary',
                                            batch_size = BATCH,
                                            seed = SEED)

ds_test = val_datagen.flow_from_dataframe(df_test,
                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'binary',
                                            batch_size = 1,
                                            shuffle = False)


ds_train_efficient = train_datagen.flow_from_dataframe(train_df,
                                             x_col = 'image',
                                             y_col = 'class',
                                             target_size = (IMG_SIZE_EFF, IMG_SIZE_EFF),
                                             class_mode = 'binary',
                                             batch_size = BATCH,
                                             seed = SEED)

ds_val_efficient = val_datagen.flow_from_dataframe(val_df,
                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE_EFF, IMG_SIZE_EFF),
                                            class_mode = 'binary',
                                            batch_size = BATCH,
                                            seed = SEED)

ds_test_efficient = val_datagen.flow_from_dataframe(df_test,
                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE_EFF, IMG_SIZE_EFF),
                                            class_mode = 'binary',
                                            batch_size = 1,
                                            shuffle = False)

In [ ]:
# retrieve a batch of images from the data generator
x_batch, y_batch = next(ds_train)
# plot the first 9 images in the batch
for i in range(32):
    # define subplot
    plt.subplot(4,8,i + 1)
    # plot raw pixel data
    plt.imshow(x_batch[i])
# show the figure
plt.show()

## class weights

In [ ]:
class_weights_arr = class_weight.compute_class_weight('balanced',
                                                 classes=np.unique(ds_train.classes), y=ds_train.classes)


if oversampling_flag:   #use the previously calculated values
    class_weights={0: 1.9440820130475303, 1: 0.6731203614069055}  
else:
    class_weights_arr[1]
    class_weights={0: class_weights_arr[0], 1: class_weights_arr[1]}                                        
    print(class_weights)

Now, we are ready for the next stage: creating and training the image classification models.

# <a id="5">Transfer Learning</a> 

The approach, called transfer learning, consists of using a pretrained model as a feature extractor. In this notebook, the selected model was the ResNet152V2 available on the Keras Package [(link)](https://keras.io/api/applications/resnet/#resnet152v2-function). 

This model was already trained in another dataset (ImageNet). What we do here is to set include_top to false, removing the ‘head’, responsible for assigning the classes in this other dataset, and keep all the previous layers. Then, we include our last few layers, including the one responsible for generating the output.



In [ ]:
base_model_xception = tf.keras.applications.Xception(
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False, 
     classes = 2)

base_model_resnet = tf.keras.applications.ResNet152V2(
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False)

base_model_efficientnet = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE_EFF, IMG_SIZE_EFF, 3),
    classes=2,
    classifier_activation="softmax"
)

base_model_xception.trainable = False
base_model_resnet.trainable = False
base_model_efficientnet.trainable = False

def get_pretrained(architecture):
    
    #Input shape = [width, height, color channels]
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    if architecture.lower() == "xception":
        x = base_model_xception(inputs)
    elif architecture.lower() == "efficientnet":
        inputs = layers.Input(shape=(IMG_SIZE_EFF, IMG_SIZE_EFF, 3))
        x = base_model_efficientnet(inputs)
    elif architecture.lower() == "resnet":
        x = base_model_resnet(inputs)

    # Head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.1)(x)
    
    #Final Layer (Output)
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs=[inputs], outputs=output)
    
    return model

In [ ]:
#Setting callbakcs

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=1e-7,
    restore_best_weights=True,
)

plateau = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor = 0.2,                                     
    patience = 4,                                   
    min_delt = 1e-7,                                
    cooldown = 0,                               
    verbose = 1
) 

# define the ModelCheckpoint callback

checkpoint_path_efficientnet = "model_checkpoint_efficientnet.h5"
checkpoint_callback_efficientnet = ModelCheckpoint(filepath=checkpoint_path_efficientnet,
                                      save_weights_only=True,
                                      monitor='val_loss',
                                      save_best_only=True)

checkpoint_path_resnet = "model_checkpoint_resnet.h5"
checkpoint_callback_resnet = ModelCheckpoint(filepath=checkpoint_path_resnet,
                                      save_weights_only=True,
                                      monitor='val_loss',
                                      save_best_only=True)

checkpoint_path_xception = "model_checkpoint_xception.h5"
checkpoint_callback_xception = ModelCheckpoint(filepath=checkpoint_path_xception,
                                      save_weights_only=True,
                                      monitor='val_loss',
                                      save_best_only=True)

We now print the different models we are going to be comparing:

In [ ]:
keras.backend.clear_session()

model_pretrained_xception = get_pretrained("xception")
model_pretrained_xception.compile(loss='binary_crossentropy'
              , optimizer = keras.optimizers.Adam(learning_rate=5e-4), metrics='binary_accuracy')

model_pretrained_xception.summary()

In [ ]:
keras.backend.clear_session()

model_pretrained_resnet = get_pretrained("resnet")
model_pretrained_resnet.compile(loss='binary_crossentropy'
              , optimizer = keras.optimizers.Adam(learning_rate=5e-4), metrics='binary_accuracy')

model_pretrained_resnet.summary()

In [ ]:
keras.backend.clear_session()

model_pretrained_efficientnet = get_pretrained("efficientnet")
model_pretrained_efficientnet.compile(loss='binary_crossentropy'
              , optimizer = keras.optimizers.Adam(learning_rate=5e-4), metrics='binary_accuracy')

model_pretrained_efficientnet.summary()

Now we train each model

In [ ]:
with tf.device("/GPU:0"):
    history_xception = model_pretrained_xception.fit(ds_train,
          batch_size = BATCH, epochs = epos,
          validation_data=ds_val,
          callbacks=[early_stopping, plateau, checkpoint_callback_xception],
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH),
          class_weight=class_weights);
    
    history_resnet = model_pretrained_resnet.fit(ds_train,
          batch_size = BATCH, epochs = epos,
          validation_data=ds_val,
          callbacks=[early_stopping, plateau, checkpoint_callback_resnet],
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH),
          class_weight=class_weights);
    
    history_efficientnet = model_pretrained_efficientnet.fit(ds_train_efficient,
          batch_size = BATCH, epochs = epos,
          validation_data=ds_val_efficient,
          callbacks=[early_stopping, plateau, checkpoint_callback_efficientnet],
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH),
          class_weight=class_weights);

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))

sns.lineplot(x = history_resnet.epoch, y = history_resnet.history['loss'], label = "Resnet Training loss", marker = "o", color = "blue", markersize = 10)
sns.lineplot(x = history_resnet.epoch, y = history_resnet.history['val_loss'], label = "Resnet Validation Loss", marker = "o", color = "red", markersize = 10)

sns.lineplot(x = history_xception.epoch, y = history_xception.history['loss'], label = "Xception Training loss", marker = "*", color = "blue", markersize = 10)
sns.lineplot(x = history_xception.epoch, y = history_xception.history['val_loss'], label = "Xception Validation Loss", marker = "*", color = "red", markersize = 10)

sns.lineplot(x = history_efficientnet.epoch, y = history_efficientnet.history['loss'], label = "Efficientnet Training loss", marker = "X", color = "blue", markersize = 10)
sns.lineplot(x = history_efficientnet.epoch, y = history_efficientnet.history['val_loss'], label = "Efficientnet Validation Loss", marker = "X", color = "red", markersize = 10)

ax.set_title('Learning Curve (Loss)')
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
ax.set_ylim(0.05, 0.9)
ax.legend()
plt.savefig("Loss_curve_before_fine-tuning.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))


sns.lineplot(x = history_resnet.epoch, y = history_resnet.history['binary_accuracy'], label = "Resnet Binary Accuracy", marker = "o", color = "blue", markersize = 10)
sns.lineplot(x = history_resnet.epoch, y = history_resnet.history['val_binary_accuracy'], label = "Resnet Validation Binary Accuracy", marker = "o", color = "red", markersize = 10)

sns.lineplot(x = history_xception.epoch, y = history_xception.history['binary_accuracy'], label = "Xception Training Binary Accuracy", marker = "*", color = "blue", markersize = 10)
sns.lineplot(x = history_xception.epoch, y = history_xception.history['val_binary_accuracy'], label = "Xception Validation Binary Accuracy", marker = "*", color = "red", markersize = 10)

sns.lineplot(x = history_efficientnet.epoch, y = history_efficientnet.history['binary_accuracy'], label = "Efficientnet Training Binary Accuracy", marker = "X", color = "blue", markersize = 10)
sns.lineplot(x = history_efficientnet.epoch, y = history_efficientnet.history['val_binary_accuracy'], label = "Efficientnet Validation Binary Accuracy", marker = "X", color = "red", markersize = 10)


ax.set_title('Learning Curve (Accuracy)')
ax.set_ylabel('Accuracy')
ax.set_xlabel('Epoch')
ax.set_ylim(0.7, 1.0)
ax.legend()
plt.savefig("Accuracy_curve_before_fine-tuning.pdf")
plt.show()

### Now we print the acccuracy of the models:

In [ ]:
val_accuracies = []

In [ ]:
score = model_pretrained_xception.evaluate(ds_val, steps = len(val_df)/BATCH, verbose = 0)
print("Scores for the Xception model:")
print('Val loss:', score[0])
print('Val accuracy:', score[1])
val_accuracies.append(score[1])

In [ ]:
score = model_pretrained_resnet.evaluate(ds_val, steps = len(val_df)/BATCH, verbose = 0)
print("Scores for the resnet model:")
print('Val loss:', score[0])
print('Val accuracy:', score[1])
val_accuracies.append(score[1])

In [ ]:
score = model_pretrained_efficientnet.evaluate(ds_val_efficient, steps = len(val_df)/BATCH, verbose = 0)
print("Scores for the efficientnet model:")
print('Val loss:', score[0])
print('Val accuracy:', score[1])
val_accuracies.append(score[1])

### And we now evalute all models:

In [ ]:
test_accuracies = []
score = model_pretrained_xception.evaluate(ds_test, steps = len(df_test), verbose = 0)
print("Evaluation for the for the xception model:")
print('Test loss:', score[0])
print('Test accuracy:', score[1])
test_accuracies.append(score[1])

In [ ]:
score = model_pretrained_resnet.evaluate(ds_test, steps = len(df_test), verbose = 0)
print("Evaluation for the for the resnet model:")
print('Test loss:', score[0])
print('Test accuracy:', score[1])
test_accuracies.append(score[1])

In [ ]:
score = model_pretrained_efficientnet.evaluate(ds_test_efficient, steps = len(df_test), verbose = 0)
print("Evaluation for the for the efficientnet model:")
print('Test loss:', score[0])
print('Test accuracy:', score[1])
test_accuracies.append(score[1])

In [ ]:
before_fine_tuning = {"Model":["Xception", "Resnet", "Efficientnet"], "Validation Accuracy":val_accuracies, "Test Accuracy":test_accuracies}
before_fine_tuning = pd.DataFrame(before_fine_tuning)



In [ ]:
before_fine_tuning

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    name='Validation Accuracy',
    x=['Xception', 'Resnet', 'Efficientnet'], y=val_accuracies, text = val_accuracies))
fig.add_trace(go.Bar(
    name='Test Accuracy',
    x=['Xception', 'Resnet', 'Efficientnet'], y=test_accuracies, text=test_accuracies
))
fig.update_layout(
    title="Accuracies before fine tuning the models",
    xaxis_title="Model",
    yaxis_title="Arbitrary units",
    legend_title="Dataset",
    barmode='group'
)

fig.show()
fig.write_image("Accuracies_before_fine-tuning.pdf")

# <a id="7">Fine Tuning</a> 

Our last approach is called Fine Tuning. In the last section, all the layers from the pretrained model were ‘frozen’, preserving the weights calculated during its training on the ImageNet dataset. Now, we are going to unfreeze a few of its last layers and continue the training, tuning the weights from these layers according to our dataset.

In [ ]:
base_model_xception.trainable = True
base_model_resnet.trainable = True
base_model_efficientnet.trainable = True

# Unfreeze all layers except for the last 13
#for layer in base_model.layers[:-13]:
#    layer.trainable = False

unfreeze_xception = 2 #Highest numbers
unfreeze_resnet = 3
unfreeze_efficientnet = 4

### We unfreeze the models:

count = 0    
for layer in base_model_xception.layers[::-1]: #We reverse the array
    if ("conv" in layer.name) and (count < unfreeze_xception):
        count += 1
    else:
        layer.trainable = False

In [ ]:
"""
Unfreezing for xception layers
"""

for layer in base_model_xception.layers[:-unfreeze_xception]:
    layer.trainable = False

count = 0    
for layer in base_model_resnet.layers[::-1]: #We reverse the array
    if ("conv" in layer.name) and (count < unfreeze_resnet):
        count += 1
    else:
        layer.trainable = False

In [ ]:
"""
Unfreezing for resnet layers
"""

for layer in base_model_resnet.layers[:-unfreeze_resnet]:
    layer.trainable = False

count = 0    
for layer in base_model_efficientnet.layers[::-1]: #We reverse the array
    if ("conv" in layer.name) and (count < unfreeze_efficientnet):
        count += 1
    else:
        layer.trainable = False

In [ ]:
"""
Unfreezing for efficientnet layers
"""

for layer in base_model_efficientnet.layers[:-unfreeze_efficientnet]:
    layer.trainable = False

### And now we we see the summary of these new models to be trained:

In [ ]:
model_pretrained_xception = get_pretrained("xception")
model_pretrained_xception.compile(loss='binary_crossentropy'
              , optimizer = keras.optimizers.Adam(learning_rate=2e-5), metrics='binary_accuracy')

print("Summary of Xception model:")
model_pretrained_xception.summary()

In [ ]:
model_pretrained_resnet = get_pretrained("resnet")
model_pretrained_resnet.compile(loss='binary_crossentropy'
              , optimizer = keras.optimizers.Adam(learning_rate=2e-5), metrics='binary_accuracy')

print("Summary of Resnet model:")
model_pretrained_resnet.summary()

In [ ]:
model_pretrained_efficientnet = get_pretrained("efficientnet")
model_pretrained_efficientnet.compile(loss='binary_crossentropy'
              , optimizer = keras.optimizers.Adam(learning_rate=2e-5), metrics='binary_accuracy')

print("Summary of Efficientnet model:")
model_pretrained_efficientnet.summary()

### Now we train the model:

In [ ]:
with tf.device("/GPU:0"):
    history_xception = model_pretrained_xception.fit(ds_train,
          batch_size = BATCH, epochs = epos,
          validation_data=ds_val,
          callbacks=[early_stopping, plateau, checkpoint_callback_xception],
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH),
          class_weight=class_weights);

In [ ]:
with tf.device("/GPU:0"):
    history_resnet = model_pretrained_resnet.fit(ds_train,
          batch_size = BATCH, epochs = epos,
          validation_data=ds_val,
          callbacks=[early_stopping, plateau, checkpoint_callback_resnet],
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH),
          class_weight=class_weights);

In [ ]:
with tf.device("/GPU:0"):    
    history_efficientnet = model_pretrained_efficientnet.fit(ds_train_efficient,
          batch_size = BATCH, epochs = epos,
          validation_data=ds_val_efficient,
          callbacks=[early_stopping, plateau, checkpoint_callback_efficientnet],
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH),
          class_weight=class_weights);

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))

sns.lineplot(x = history_resnet.epoch, y = history_resnet.history['loss'], label = "Resnet Training loss", marker = "o", color = "blue", markersize = 10)
sns.lineplot(x = history_resnet.epoch, y = history_resnet.history['val_loss'], label = "Resnet Validation Loss", marker = "o", color = "red", markersize = 10)

sns.lineplot(x = history_xception.epoch, y = history_xception.history['loss'], label = "Xception Training loss", marker = "*", color = "blue", markersize = 10)
sns.lineplot(x = history_xception.epoch, y = history_xception.history['val_loss'], label = "Xception Validation Loss", marker = "*", color = "red", markersize = 10)

sns.lineplot(x = history_efficientnet.epoch, y = history_efficientnet.history['loss'], label = "Efficientnet Training loss", marker = "X", color = "blue", markersize = 10)
sns.lineplot(x = history_efficientnet.epoch, y = history_efficientnet.history['val_loss'], label = "Efficientnet Validation Loss", marker = "X", color = "red", markersize = 10)

ax.set_title('Learning Curve (Loss)')
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
ax.set_ylim(0.05, 0.9)
ax.legend()
plt.savefig("Loss_curve_after_fine-tuning.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))


sns.lineplot(x = history_resnet.epoch, y = history_resnet.history['binary_accuracy'], label = "Resnet Binary Accuracy", marker = "o", color = "blue", markersize = 10)
sns.lineplot(x = history_resnet.epoch, y = history_resnet.history['val_binary_accuracy'], label = "Resnet Validation Binary Accuracy", marker = "o", color = "red", markersize = 10)

sns.lineplot(x = history_xception.epoch, y = history_xception.history['binary_accuracy'], label = "Xception Training Binary Accuracy", marker = "*", color = "blue", markersize = 10)
sns.lineplot(x = history_xception.epoch, y = history_xception.history['val_binary_accuracy'], label = "Xception Validation Binary Accuracy", marker = "*", color = "red", markersize = 10)

sns.lineplot(x = history_efficientnet.epoch, y = history_efficientnet.history['binary_accuracy'], label = "Efficientnet Training Binary Accuracy", marker = "X", color = "blue", markersize = 10)
sns.lineplot(x = history_efficientnet.epoch, y = history_efficientnet.history['val_binary_accuracy'], label = "Efficientnet Validation Binary Accuracy", marker = "X", color = "red", markersize = 10)


ax.set_title('Learning Curve (Accuracy)')
ax.set_ylabel('Accuracy')
ax.set_xlabel('Epoch')
ax.set_ylim(0.7, 1.0)
ax.legend()
plt.savefig("Accuracy_curve_after_fine-tuning.pdf")
plt.show()

### Now we print the acccuracy of the models again:

In [ ]:
val_accuracies_after = []

In [ ]:
score_xception = model_pretrained_xception.evaluate(ds_val, steps = len(val_df)/BATCH, verbose = 0)
print("Scores for the Xception model:")
print('Val loss:', score_xception[0])
print('Val accuracy:', score_xception[1])
val_accuracies_after.append(score_xception[1])

In [ ]:
score_resnet = model_pretrained_resnet.evaluate(ds_val, steps = len(val_df)/BATCH, verbose = 0)
print("Scores for the resnet model:")
print('Val loss:', score_resnet[0])
print('Val accuracy:', score_resnet[1])
val_accuracies_after.append(score_resnet[1])

In [ ]:
score_efficientnet = model_pretrained_efficientnet.evaluate(ds_val_efficient, steps = len(val_df)/BATCH, verbose = 0)
print("Scores for the efficientnet model:")
print('Val loss:', score_efficientnet[0])
print('Val accuracy:', score_efficientnet[1])
val_accuracies_after.append(score_efficientnet[1])

### And we now evalute all models:

In [ ]:
test_accuracies_after = []
score_xception = model_pretrained_xception.evaluate(ds_test, steps = len(df_test), verbose = 0)
print("Evaluation for the for the xception model:")
print('Test loss:', score_xception[0])
print('Test accuracy:', score_xception[1])
test_accuracies_after.append(score_xception[1])

In [ ]:
score_resnet = model_pretrained_resnet.evaluate(ds_test, steps = len(df_test), verbose = 0)
print("Evaluation for the for the resnet model:")
print('Test loss:', score_resnet[0])
print('Test accuracy:', score_resnet[1])
test_accuracies_after.append(score_resnet[1])

In [ ]:
score_efficientnet = model_pretrained_efficientnet.evaluate(ds_test_efficient, steps = len(df_test), verbose = 0)
print("Evaluation for the for the efficientnet model:")
print('Test loss:', score_efficientnet[0])
print('Test accuracy:', score_efficientnet[1])
test_accuracies_after.append(score_efficientnet[1])

In [ ]:
after_fine_tuning = {"Model":["Xception", "Resnet", "Efficientnet"], "Validation Accuracy":val_accuracies_after,
                      "Test Accuracy":test_accuracies_after, 
                     "Unfrozen layers":[unfreeze_xception, unfreeze_resnet, unfreeze_efficientnet]}
after_fine_tuning = pd.DataFrame(after_fine_tuning)



In [ ]:
after_fine_tuning

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    name='Validation Accuracy',
    x=['Xception', 'Resnet', 'Efficientnet'], y=val_accuracies_after, text = val_accuracies_after))
fig.add_trace(go.Bar(
    name='Test Accuracy',
    x=['Xception', 'Resnet', 'Efficientnet'], y=test_accuracies_after, text = test_accuracies_after
))
fig.update_layout(
    title="Accuracies After fine tuning the models",
    xaxis_title="Model",
    yaxis_title="Arbitrary units",
    legend_title="Dataset",
    barmode='group'
)

fig.show()
fig.write_image("Accuracies_after_fine_tuning.pdf")

As expected, the fine-tuning approach has reached the best score. We end this notebook by showing a few performance metrics.

# <a id="8">Performance Metrics</a> 

### We now test all of our models:

In [ ]:
num_label = {'Normal': 0, 'Pneumonia' : 1}
Y_test = df_test['class'].copy().map(num_label).astype('int')

In [ ]:
ds_test.reset()
ds_test_efficient.reset()

predictions_xception = model_pretrained_xception.predict(ds_test, steps=len(ds_test), verbose=0)
predictions_resnet = model_pretrained_resnet.predict(ds_test, steps=len(ds_test), verbose=0)
predictions_efficientnet = model_pretrained_efficientnet.predict(ds_test_efficient, steps=len(ds_test), verbose=0)


pred_labels_xception = np.where(predictions_xception >0.5, 1, 0)
pred_labels_resnet = np.where(predictions_resnet >0.5, 1, 0)
pred_labels_efficientnet = np.where(predictions_efficientnet >0.5, 1, 0)

In [ ]:
print("Xception Accuracy: ", accuracy_score(Y_test, pred_labels_xception))
print("Resnet Accuracy: ", accuracy_score(Y_test, pred_labels_resnet))
print("Efficientnet Accuracy: ", accuracy_score(Y_test, pred_labels_efficientnet))

### Visualizing the confusion matrix:

In [ ]:
confusion_matrix = metrics.confusion_matrix(Y_test, pred_labels_xception)
sns.heatmap(confusion_matrix, annot=True, fmt="d")

plt.title("Confusion Matrix for Xception")
plt.xlabel("Predicted Label", fontsize= 12)
plt.ylabel("True Label", fontsize= 12)

plt.savefig("Xception_confusion_matrix.pdf")
plt.show()

In [ ]:
confusion_matrix = metrics.confusion_matrix(Y_test, pred_labels_resnet)
sns.heatmap(confusion_matrix, annot=True, fmt="d")

plt.title("Confusion Matrix for Resnet")
plt.xlabel("Predicted Label", fontsize= 12)
plt.ylabel("True Label", fontsize= 12)

plt.savefig("Resnet_confusion_matrix.pdf")
plt.show()

In [ ]:
confusion_matrix = metrics.confusion_matrix(Y_test, pred_labels_efficientnet)
sns.heatmap(confusion_matrix, annot=True, fmt="d")

plt.title("Confusion Matrix for Efficientnet")
plt.xlabel("Predicted Label", fontsize= 12)
plt.ylabel("True Label", fontsize= 12)

plt.savefig("Efficientnet_confusion_matrix.pdf")
plt.show()

### Metrics classification report for all models:

In [ ]:
print(metrics.classification_report(Y_test, pred_labels_xception, labels = [0, 1]))

In [ ]:
print(metrics.classification_report(Y_test, pred_labels_resnet, labels = [0, 1]))

In [ ]:
print(metrics.classification_report(Y_test, pred_labels_efficientnet, labels = [0, 1]))

### And finally, we plot the ROC curve:

In [ ]:
roc_auc_xception = metrics.roc_auc_score(Y_test, predictions_xception)
roc_auc_resnet = metrics.roc_auc_score(Y_test, predictions_resnet)
roc_auc_efficientnet = metrics.roc_auc_score(Y_test, predictions_efficientnet)

print('ROC_AUC Xception: ', roc_auc_xception)
print('ROC_AUC Resnet: ', roc_auc_resnet)
print('ROC_AUC Efficientnet: ', roc_auc_efficientnet)

fpr_xception, tpr_xception, thresholds_xception = metrics.roc_curve(Y_test, predictions_xception)
fpr_resnet, tpr_resnet, thresholds_resnet = metrics.roc_curve(Y_test, predictions_resnet)
fpr_efficientnet, tpr_efficientnet, thresholds_efficientnet = metrics.roc_curve(Y_test, predictions_efficientnet)

plt.plot(fpr_xception, tpr_xception, label = 'Xception ROC_AUC = %0.3f' % roc_auc_xception)
plt.plot(fpr_resnet, tpr_resnet, label = 'Resnet ROC_AUC = %0.3f' % roc_auc_resnet)
plt.plot(fpr_efficientnet, tpr_efficientnet, label = 'Efficientnet ROC_AUC = %0.3f' % roc_auc_efficientnet)

plt.xlabel("False Positive Rate", fontsize= 12)
plt.ylabel("True Positive Rate", fontsize= 12)
plt.legend(loc="lower right")
plt.title("ROC Curve after training additional layers")

plt.savefig("ROC_Curve_after_training_additional_layers.pdf")
plt.show()

The recall was close to 100%. Even without expertise on the medical field, it’s reasonable to assume that false negatives are more ‘costly’ than false positives in this case. Reaching such recall with a relatively small dataset for training as this one, while also reaching a pretty good recall, is a good indicative of the model’s capabilities. Such capabilities are also confirmed by the high ROC-AUC value.

In [ ]:
print("Total elapsed time:", time.time()- initial_time)

In [ ]:
after_fine_tuning

In [ ]:
before_fine_tuning

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    name='Validation Accuracy Before Fine Tuning', marker = {"color":"blue"},
    x=['Xception', 'Resnet', 'Efficientnet'], y=val_accuracies, text = val_accuracies))
fig.add_trace(go.Bar(
    name='Validation Accuracy After Fine Tuning',marker = {"color":"lightslategrey"},
    x=['Xception', 'Resnet', 'Efficientnet'], y=val_accuracies_after, text = val_accuracies_after
))
fig.update_layout(
    title="Validation accuracies comparison after and before fine tuning the models",
    xaxis_title="Model",
    yaxis_title="Arbitrary units",
    legend_title="Dataset",
    barmode='group'
)

fig.show()
fig.write_image("Accuracies_before_and_after_fine_tuning.pdf")

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    name='Validation Accuracy Before Fine Tuning', marker = {"color":"red"},
    x=['Xception', 'Resnet', 'Efficientnet'], y=test_accuracies, text = test_accuracies))
fig.add_trace(go.Bar(
    name='Test Accuracy After Fine Tuning',marker = {"color":"orange"},
    x=['Xception', 'Resnet', 'Efficientnet'], y=test_accuracies_after, text = test_accuracies_after
))
fig.update_layout(
    title="Test accuracies comparison after and before fine tuning the models",
    xaxis_title="Model",
    yaxis_title="Arbitrary units",
    legend_title="Dataset",
    barmode='group'
)

fig.show()
fig.write_image("Test_Accuracies_before_and_after_fine_tuning.pdf")